# Recurrent neural network(RNN) 


Recurrent neural network(RNN)은 Text, 음성, 시계열 데이터등 순차적인(Sequential) 데이터들에 대한 분석을 할때 주로 사용하는 알고리즘입니다. <br>
본 문서에서는 RNN의 기본적인 내용부터 실습하겠습니다.



In [9]:
#-*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint #데이터를 보기 좋게 출력해주는 모듈

AttributeError: module 'pandas' has no attribute 'computation'

### Text 데이터를 어떻게 Neural Network에 넣을까?
Text 데이터를 Neural Network에 넣는 방법은 벡터화(embedding)입니다.<br>
Text 데이터를 벡터화 시키는 방법은 여러가지(One-hot, bag of words, tf-idf, word embedding)가 있습니다.<br>
오늘은 그중 하나인 one-hot representation을 사용하겠습니다.


In [4]:
# 각 문자를 벡터로 나타내기 위해 one-hot representation 형태로 나타냅니다.
# I LOVE YOU 를 표현하기 위해 문자를 One-hot represnetation으로 나타내었습니다.
I = [1, 0, 0, 0, 0, 0, 0]

L = [0, 1, 0, 0, 0, 0, 0]
O = [0, 0, 1, 0, 0, 0, 0]
V = [0, 0, 0, 1, 0, 0, 0]
E = [0, 0, 0, 0, 1, 0, 0]

Y = [0, 0, 0, 0, 0, 1, 0]
U = [0, 0, 0, 0, 0, 0, 1]

In [5]:
sess = tf.InteractiveSession()

### RNN 과 LSTM
Tensorflow에서는 쉽게 RNN을 정의 할 수 있습니다. 아래에서는 편의상 함수의 형태로 정의하였습니다.<br>
아래 선언된 RNN cell중 LSTM(Long Short-Term Memory)은 RNN의 Vanishing gradient 문제를 해결하고 이전 정보를 더욱 잘 반영하기 위해서 RNN의 Activation unit을 변경한 것입니다.
![lstm](./assets/lstm.png)

In [6]:
def RNN(x_input, sequence_length, rnn_type='rnn'):
    with tf.variable_scope('rnn') as scope:
        
        hidden_size = 2 # RNN의 Hidden Layer의 차원을 의미합니다.
        
        cell = None
        if(rnn_type == 'rnn'):
            cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
        elif(rnn_type == 'lstm'):
            cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
            #state_is_tuple = True로 하면 State의 값을 tuple형태로 분리해서 보여줍니다.
                                    
        outputs, states = tf.nn.dynamic_rnn(cell, x_input,
                                            sequence_length=sequence_length, #[8], ILOVEYOU의 길이
                                            dtype=tf.float32)
        sess.run(tf.global_variables_initializer())
        print("outputs of RNN")
        print(sess.run(outputs))
        print("states of RNN")
        pprint.pprint(sess.run(states))

### TensorFlow에서 데이터의 가장 첫번째 차원은 Batch_size !
Tensorflow에서 데이터의 가장 첫번째 Dimension은 Batch에 해당하기 때문에 아래와 같이 사용할 데이터인<br>
1차원에 속한 [I, L, O, V, E, Y, O, U]를 -> 2차원인 [[I, L, O, V, E, Y, O, U]]로 할당해 줍니다.<br>
현재 차원 (batch_size, sequence_length, one_hot_representation_dimension) == (1, 8, 7)

In [7]:
x_input = np.array([[I, L, O, V, E, Y, O, U]], dtype=np.float32) 
print("x_input.shape")
print(x_input.shape)
print("x_input")
print(x_input)
sequence_length = x_input.shape[1] #글자 길이
sequence_length = [sequence_length] # 첫번째 차원이 Batch기 때문에 실수가 아닌 []로 변환

x_input.shape
(1, 8, 7)
x_input
[[[ 1.  0.  0.  0.  0.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  1.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  0.]
  [ 0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  1.]]]


### 선언한 RNN함수를 호출
RNN함수를 호출합니다. <br>
RNN의 각 Time step 별로 어떤 Output이 나오는지, RNN의 State값은 무엇인지를 확인합니다.

In [8]:
RNN(x_input, sequence_length, rnn_type='rnn')

AttributeError: module 'pandas' has no attribute 'computation'

### Dynamic_RNN
텐서플로우에서는 Static_RNN과 Dynamic_RNN이 있습니다. <br>
그러나 거의 대부분 Dynamic_RNN을 사용합니다. Dynamic_RNN은 Input data의 길이(sequence)를 고려하여 계산이 가능합니다.<br>
아래 코드에서 길이가 8인 문자열에 대해서 길이 7만큼만 계산하면 결과가 어떻게 나오는지 확인해보시면 7이상인 경우 Output값이 0이 나옴을 확인할 수 있습니다.

In [7]:
# Sequence Length를 3으로 변경하면 Dynamic RNN의 특성상
# Time Step 3번째 까지만 계산합니다.
#tf.get_variable_scope().reuse_variables() # 같은 RNN을 재사용할때 필요한 코드
x_input_2 = np.array([[I, L, O, V, E, Y, O, U],
                      [I, L, O, V, E, Y, O, U]], dtype=np.float32) 
RNN(x_input_2, sequence_length=[8,7], rnn_type='lstm')

outputs of RNN
[[[-0.19231324 -0.05708623]
  [-0.04951089 -0.0964755 ]
  [-0.14186911 -0.11884118]
  [-0.21588489 -0.29764304]
  [-0.19670735 -0.12379649]
  [-0.20099474 -0.16918392]
  [-0.30278811 -0.18010156]
  [-0.29759625 -0.29654554]]

 [[-0.19231324 -0.05708623]
  [-0.04951089 -0.0964755 ]
  [-0.14186911 -0.11884118]
  [-0.21588489 -0.29764304]
  [-0.19670735 -0.12379649]
  [-0.20099474 -0.16918392]
  [-0.30278811 -0.18010156]
  [ 0.          0.        ]]]
states of RNN
LSTMStateTuple(c=array([[-0.5437721 , -0.71608835],
       [-0.63887435, -0.49504983]], dtype=float32), h=array([[-0.29759625, -0.29654554],
       [-0.30278811, -0.18010156]], dtype=float32))


"I LOVE YOU" 문장의 길이가 총 8이므로 RNN의 Output도 각 Time Step별로 총 8개가 나옵니다. <br>RNN의 State는 같은 Neural Network를 반복적으로 쓰는 RNN의 특성 때문에 Time Step과 상관없이 하나의 값이 나오게 됩니다.

## Multi Layer RNN
RNN을 여러개 쌓을 수도 있습니다. Tensorflow에서는 tf.contrib.rnn.MultiRNNCell이라는 API를 제공하며 아래와 같이 Multi Layer RNN을 구성할 수 있습니다.

In [8]:
def Multi_RNN(x_input, sequence_length, rnn_type='rnn'):
    with tf.variable_scope('multi_rnn') as scope:
        
        hidden_size = 2
        num_layers = 2
        
        cell = None
        if(rnn_type == 'rnn'):
            cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
        elif(rnn_type == 'lstm'):
            cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
            #state_is_tuple = True로 하면 Cell State와 Hidden State의 값을 tuple형태로 분리해서 보여줍니다.
        cell = tf.contrib.rnn.MultiRNNCell(cells=[cell] * num_layers, state_is_tuple=True)
        outputs, states = tf.nn.dynamic_rnn(cell, x_input,
                                            sequence_length=sequence_length,
                                            dtype=tf.float32)
        sess.run(tf.global_variables_initializer())
        print("outputs of Multi_RNN")
        print(sess.run(outputs))
        print("states of Multi_RNN")
        pprint.pprint(sess.run(states))

Multi Layer RNN의 경우 Output값은 가장 위에 층의 값을 사용하게 됩니다.
State는 각 층당 1개씩 출력해주는걸 볼 수 있습니다.

In [9]:
Multi_RNN(x_input, sequence_length, rnn_type='lstm')

outputs of Multi_RNN
[[[ 0.00077869  0.00561604]
  [-0.02991019  0.01418542]
  [-0.06328127  0.02125617]
  [-0.06087604  0.02229038]
  [-0.05649587  0.01666107]
  [-0.07129844  0.020167  ]
  [-0.09387885  0.02434255]
  [-0.10773665  0.01925956]]]
states of Multi_RNN
(LSTMStateTuple(c=array([[ 0.19464058, -0.54733354]], dtype=float32), h=array([[ 0.07342119, -0.22338189]], dtype=float32)),
 LSTMStateTuple(c=array([[-0.21893743,  0.03750154]], dtype=float32), h=array([[-0.10773665,  0.01925956]], dtype=float32)))


## Bidirectional RNN
빈칸 넣기 게임처럼 문맥을 고려해야 해결할 수 있을 때가 있습니다. <br>
Bidirectional RNN은 이러한 경우를 고려한 뉴럴 네트워크입니다. <br>
Tensorflow에서는 tf.nn.bidirectional_dynamic_rnn이라는 API를 통해 손쉽게 Bidirectional RNN을 구성할 수 있습니다. <br>

### Tip
Bidirectional RNN에 데이터가 들어가는 과정에 대해서 조금 더 자세히 말씀드리자면, Input 데이터의 순서를 '반대'로 바꿔서 집어 넣는다고 생각하시면 됩니다. 이러한 과정은 텐서플로우 내에서 자동으로 이루어 집니다.

In [10]:
def Bi_RNN(x_input, sequence_length, rnn_type='rnn'):
    with tf.variable_scope('bidirectional_rnn') as scope:
        
        hidden_size = 2
        num_layers = 2
        
        cell_fw = None
        cell_bw = None
        if(rnn_type == 'rnn'):
            cell_fw = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
            cell_bw = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
        elif(rnn_type == 'lstm'):
            cell_fw = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
            cell_bw = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
            #state_is_tuple = True로 하면 Cell State와 Hidden State의 값을 tuple형태로 분리해서 보여줍니다.
            
        outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_input, 
                                                          sequence_length=sequence_length,
                                                          dtype=tf.float32)
        sess.run(tf.global_variables_initializer())
        print("outputs of Multi_RNN")
        print(sess.run(outputs))
        print("states of Multi_RNN")
        pprint.pprint(sess.run(states))

In [11]:
Bi_RNN(x_input, sequence_length, rnn_type='lstm')

outputs of Multi_RNN
(array([[[-0.06758793, -0.01852215],
        [-0.16376032, -0.03272057],
        [-0.09567963, -0.06403921],
        [-0.11466568, -0.09637608],
        [ 0.06649007,  0.03735987],
        [-0.05943045, -0.01769379],
        [ 0.00136965, -0.04845091],
        [ 0.01521786, -0.15079093]]], dtype=float32), array([[[ 0.1013485 , -0.06081938],
        [ 0.03055528, -0.02659458],
        [ 0.02441726,  0.02000265],
        [-0.01413904,  0.01390083],
        [-0.11112228, -0.04664689],
        [-0.02333218, -0.14173329],
        [ 0.03943175, -0.04680184],
        [-0.00178974, -0.10911496]]], dtype=float32))
states of Multi_RNN
(LSTMStateTuple(c=array([[ 0.02939724, -0.25054047]], dtype=float32), h=array([[ 0.01521786, -0.15079093]], dtype=float32)),
 LSTMStateTuple(c=array([[ 0.25206664, -0.1366279 ]], dtype=float32), h=array([[ 0.1013485 , -0.06081938]], dtype=float32)))


Output값과 State값이 데이터를 원래의 방향대로 넣어서(forward) 나온 값과 순서를 반대로 해서 넣은값(backward)값 2개로 나뉘어서 나오게 됩니다

## Multi Layer Bidirectional RNN
Bidirectional RNN을 여러층 쌓아 Multi Layer Bidirectional RNN을 만들어 보도록 하겠습니다

In [12]:
def Multi_Bi_RNN(x_input, sequence_length, rnn_type='rnn'):
    with tf.variable_scope('multi_bidirectional_rnn') as scope:
        
        hidden_size = 2
        num_layers = 2
        
        cell_fw = None
        cell_bw = None
        if(rnn_type == 'rnn'):
            cell_fw = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
            cell_bw = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
            
        elif(rnn_type == 'lstm'):
            cell_fw = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
            cell_bw = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)            
            #state_is_tuple = True로 하면 Cell State와 Hidden State의 값을 tuple형태로 분리해서 보여줍니다.
        cell_fw = tf.contrib.rnn.MultiRNNCell(cells=[cell_fw] * num_layers, state_is_tuple=True)
        cell_bw = tf.contrib.rnn.MultiRNNCell(cells=[cell_bw] * num_layers, state_is_tuple=True)
            
        outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_input, 
                                                          sequence_length=sequence_length,
                                                          dtype=tf.float32)
        sess.run(tf.global_variables_initializer())
        print("outputs of Bidirectional_Multi_RNN")
        print(sess.run(outputs))
        print("states of Bidirectional_Multi_RNN")
        pprint.pprint(sess.run(states))

In [13]:
Multi_Bi_RNN(x_input, sequence_length, rnn_type='lstm')

outputs of Bidirectional_Multi_RNN
(array([[[ 0.00756073,  0.00132258],
        [ 0.00424517, -0.02354742],
        [ 0.00700316, -0.04192907],
        [ 0.00163326, -0.06305805],
        [-0.0085106 , -0.07864803],
        [ 0.00569872, -0.06843027],
        [ 0.01326775, -0.07049034],
        [ 0.02331219, -0.05905479]]], dtype=float32), array([[[ 0.02222335, -0.0222692 ],
        [ 0.02394018, -0.02534211],
        [ 0.02030206, -0.02387164],
        [ 0.01505285, -0.01657416],
        [ 0.02100322, -0.02833175],
        [ 0.00979508, -0.01504568],
        [-0.00529743,  0.00631799],
        [-0.01081359,  0.01404889]]], dtype=float32))
states of Bidirectional_Multi_RNN
((LSTMStateTuple(c=array([[-0.24390453, -0.02500473]], dtype=float32), h=array([[-0.14028606, -0.01504857]], dtype=float32)),
  LSTMStateTuple(c=array([[ 0.04581917, -0.11993713]], dtype=float32), h=array([[ 0.02331219, -0.05905479]], dtype=float32))),
 (LSTMStateTuple(c=array([[ 0.23882382,  0.0188105 ]], dtype=floa